In [1]:
# 引入依赖
import cracknuts as cn
from cracknuts.cracker import CrackerG1

In [2]:
g1d = CrackerG1('192.168.0.10')
g1d.connect(force_update_bin=True, force_write_default_config=True)

[WARNING] 2025-12-22 11:16:46,557 cracker_basic.send_and_receive:526 Command 0x0001 with payload  received a non-OK response with status code 0x8000, and payload [Invalid clock].
[ERROR] 2025-12-22 11:16:46,561 cracker_s1.get_current_config:68 Get config from cracker error, return code 0x8000.


AttributeError: 'NoneType' object has no attribute 'glitch_vcc_normal'

In [4]:
import random
import time
from cracknuts.cracker import serial


cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"

aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

sample_length = 20000

def init(cracker):
    cracker.nut_voltage_enable()
    cracker.nut_voltage(3.3)
    cracker.nut_clock_enable()
    cracker.nut_clock_freq('8M')
    cracker.uart_io_enable()
    cracker.osc_sample_clock('48m')
    cracker.osc_sample_length(sample_length)
    cracker.osc_trigger_source('N')
    cracker.osc_analog_gain('B', 10)
    cracker.osc_trigger_level(0)
    cracker.osc_trigger_mode('E')
    cracker.osc_trigger_edge('U')
    cracker.uart_config(baudrate=serial.Baudrate.BAUDRATE_115200, bytesize=serial.Bytesize.EIGHTBITS, parity=serial.Parity.PARITY_NONE, stopbits=serial.Stopbits.STOPBITS_ONE)

    time.sleep(2)
    cmd = cmd_set_aes_enc_key + aes_key
    status, ret = cracker.uart_transmit_receive(cmd, timeout=1000, rx_count=6)

def do(cracker, count):
    plaintext_data = random.randbytes(aes_data_len)
    tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
    status, ret = cracker.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
    
    return {
        "plaintext": plaintext_data,
        "ciphertext": ret[-aes_data_len:],
        "key": bytes.fromhex(aes_key)
    }


def finish(c):
    ...
    # print('optional behavior')


acq = cn.new_acquisition(g1d, init=init, do=do, finish=finish)

cn.panel(acq)

[WARNING] 2025-12-21 16:28:44,110 cracker_s1_panel.update_cracker_panel_config:192 Failed to sync configuration to widget: the widget has no attribute named 'glitch_vcc_arm'.
[WARNING] 2025-12-21 16:28:44,111 cracker_s1_panel.update_cracker_panel_config:192 Failed to sync configuration to widget: the widget has no attribute named 'glitch_vcc_normal'.
[WARNING] 2025-12-21 16:28:44,111 cracker_s1_panel.update_cracker_panel_config:192 Failed to sync configuration to widget: the widget has no attribute named 'glitch_vcc_config_wait'.
[WARNING] 2025-12-21 16:28:44,112 cracker_s1_panel.update_cracker_panel_config:192 Failed to sync configuration to widget: the widget has no attribute named 'glitch_vcc_config_level'.
[WARNING] 2025-12-21 16:28:44,113 cracker_s1_panel.update_cracker_panel_config:192 Failed to sync configuration to widget: the widget has no attribute named 'glitch_vcc_config_count'.
[WARNING] 2025-12-21 16:28:44,113 cracker_s1_panel.update_cracker_panel_config:192 Failed to syn

In [ ]:
g1d.get_current_config()

In [ ]:
g1d.nut_voltage_enable()
g1d.nut_voltage(3.2)

In [ ]:
g1d.nut_voltage_enable()